In [1]:
!apt-get update
!apt-get install -y golang-go netcat


Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:3 https://cli.github.com/packages stable InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,153 kB]
Get:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Pa

In [2]:
%%writefile main.go
package main

import (
	"bufio"
	"fmt"
	"net"
)

var (
	clients = make(map[net.Conn]bool)
	addCh   = make(chan net.Conn)
	rmCh    = make(chan net.Conn)
	msgCh   = make(chan string)
)

func handleConnection(conn net.Conn) {
	defer conn.Close()
	addCh <- conn

	name := conn.RemoteAddr().String()
	welcome := fmt.Sprintf("Користувач %s приєднався до чату\n", name)
	msgCh <- welcome

	reader := bufio.NewScanner(conn)
	for reader.Scan() {
		text := reader.Text()
		if text == "/quit" {
			break
		}
		msg := fmt.Sprintf("%s: %s\n", name, text)
		msgCh <- msg
	}
	rmCh <- conn
	msgCh <- fmt.Sprintf("Користувач %s вийшов з чату\n", name)
}

func broadcaster() {
	for {
		select {
		case conn := <-addCh:
			clients[conn] = true
		case conn := <-rmCh:
			delete(clients, conn)
		case msg := <-msgCh:
			for c := range clients {
				fmt.Fprint(c, msg)
			}
			fmt.Print(msg) // дублюємо на сервер
		}
	}
}

func main() {
	go broadcaster()

	ln, err := net.Listen("tcp", ":9000")
	if err != nil {
		fmt.Println("Помилка запуску сервера:", err)
		return
	}
	fmt.Println("Сервер чату запущено на порту 9000")

	for {
		conn, err := ln.Accept()
		if err != nil {
			fmt.Println("Помилка підключення:", err)
			continue
		}
		go handleConnection(conn)
	}
}


Writing main.go


In [6]:
!nc 127.0.0.1 9000
